In [ ]:
import shap
import numpy as np
import matplotlib.pyplot as plt

# Assuming 'model' is your trained Keras model
input_shape = (100, 100, 1)  # Adjust if necessary

# Create a SHAP explainer
explainer = shap.GradientExplainer(model, np.zeros((1, *input_shape)))

# Select an image for explanation (example)
image = np.random.rand(1, 100, 100, 1)  # Replace with your actual image

# Compute SHAP values
shap_values = explainer.shap_values(image)

# Plot SHAP values for the segmentation output (assuming single output channel)
shap.image_plot(shap_values, -image)


In [ ]:
import shap
import numpy as np
import matplotlib.pyplot as plt

# Assuming 'model' is your trained Keras model
input_shape = (100, 100, 1)  # Adjust if necessary

# Create a SHAP explainer
explainer = shap.GradientExplainer(model, np.zeros((1, *input_shape)))

# Select an image for explanation (example)
image = np.random.rand(1, 100, 100, 1)  # Replace with your actual image

# Compute SHAP values for segmentation and classification outputs
shap_values_segmentation, shap_values_classification = explainer.shap_values(image)

# Plot SHAP values for the segmentation output
shap.image_plot(shap_values_segmentation, -image)

# Plot SHAP values for the classification output (assuming 6 classes)
for class_index in range(num_classes):
    shap.image_plot([shap_values_classification[class_index]], -image)
